In [12]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost

from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from statsmodels.graphics.tsaplots import plot_pacf
from xgboost import XGBRegressor
import lightgbm

In [13]:
all_csv = pd.read_csv('cleaned.csv.gz',
                 dtype = {
                     'store_nbr' : 'category',
                     'family' : 'category',
                     'sales': 'float',
                     'city': 'category',
                     'state': 'category',
                     'type': 'category',
                     'holiday_type': 'category',
                     'holiday_transferred': 'category'
                 },
                  parse_dates=['date'])
all_csv['date'] = pd.to_datetime(all_csv['date']).dt.to_period('D')

In [14]:
all = all_csv.copy()  # we can start experimenting from here without reloading the csv file

In [15]:
# this is for experimentation

filter_by_stores = None  # note: please use string here (unlike Mine.ipynb)
filter_by_family = None
filter_by_dates = None

#filter_by_stores = ['15']  # note: please use string here (unlike Mine.ipynb)
#filter_by_family = ['PRODUCE', 'AUTOMOBILE']
#filter_by_dates = '2014-06-05'

In [16]:
if filter_by_dates == None:
    train_start_date = '2015-06-15'
else:
    train_start_date = filter_by_dates
train_end_date = '2017-08-15'
test_start_date = '2017-08-16'
test_end_date = '2017-08-31'

In [17]:
if filter_by_family != None:
    all = all[all['family'].isin(filter_by_family)]
if filter_by_stores != None:
    all = all[all['store_nbr'].isin(filter_by_stores)]
if filter_by_dates != None:
    all = all[all['date'] >= filter_by_dates]

In [18]:
all['store_nbr'].unique()

['1', '10', '11', '12', '13', ..., '54', '6', '7', '8', '9']
Length: 54
Categories (54, object): ['1', '10', '11', '12', ..., '6', '7', '8', '9']

In [19]:
all.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3036528 entries, 0 to 3036527
Data columns (total 38 columns):
 #   Column               Dtype    
---  ------               -----    
 0   date                 period[D]
 1   store_nbr            category 
 2   family               category 
 3   sales                float64  
 4   onpromotion          int64    
 5   sales_lag_01         float64  
 6   sales_lag_02         float64  
 7   sales_lag_03         float64  
 8   sales_lag_04         float64  
 9   sales_lag_05         float64  
 10  sales_lag_06         float64  
 11  sales_lag_07         float64  
 12  sales_lag_08         float64  
 13  sales_lag_09         float64  
 14  sales_lag_10         float64  
 15  sales_lag_11         float64  
 16  sales_lag_12         float64  
 17  sales_lag_13         float64  
 18  sales_lag_14         float64  
 19  sales_lag_15         float64  
 20  sales_lag_16         float64  
 21  sales_lag_17         float64  
 22  sales_lag_18      

## One Hot Encoding

In [20]:
def one_hot_encode(df):
    ohe = OneHotEncoder()
    #ohe.fit_transform(df[column])
    return pd.get_dummies(data=df, columns=['store_nbr', 'family', 'city', 'state', 'type',
                                     'cluster', 'holiday_type', 'holiday_transferred', 'weekday'])    

In [22]:
all_ohe = one_hot_encode(all)
all_ohe = all_ohe.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  # remove bad char in column names

X = all_ohe[all_ohe['date'] <= train_end_date]
X = X.drop(['sales'], axis=1)
y = all_ohe[['date', 'sales']][all_ohe['date'] <= train_end_date]
y.set_index('date', inplace=True)

X_test = all_ohe[all_ohe['date'] >= test_start_date]
X_test = X_test.drop(['sales'], axis=1)

X.drop('date', axis=1, inplace=True)
X_test.drop('date', axis=1, inplace=True)
y.set_index(X.index, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=1)

## Experiment I -- Linear Regression

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [14]:
y_pred_train = lr.predict(X_train)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = lr.predict(X_val)
y_pred_val[y_pred_val < 0] = 0

print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))

RMS log-error train:  0.16114944543258325
RMS log-error val:  0.16183533985550788


## Experiment II -- Lightgbm

In [15]:
lgb_params = {
    'boosting_type' : 'gbdt',  # gradient boosting decision tree
    'early_stopping_rounds': 200,
    'force_col_wise': True,
    'learning_rate': 0.1,
    'max_depth': 10,
    'metric': 'mse',  # mean square error
    'num_iterations': 5000,
    'num_leaves': 10,
    'random_state': 1,
    'verbose': 0
}

X_train_lgb = lightgbm.Dataset(data=X_train, label=y_train, feature_name='auto')
X_val_lgb = lightgbm.Dataset(data=X_val, label=y_val, reference=X_train_lgb, feature_name='auto')

In [16]:
lgb = lightgbm.train(
    params=lgb_params, 
    train_set=X_train_lgb,
    valid_sets=[X_train_lgb, X_val_lgb],
)

/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


: 

: 

In [ ]:
y_pred_train = lgb.predict(X_train, num_iteration=lgb.best_iteration)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = lgb.predict(X_val, num_iteration=lgb.best_iteration)
y_pred_val[y_pred_val < 0] = 0

print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))

## Experiment III -- Random Forest (Too Slow)

In [12]:
random_forest = RandomForestRegressor(random_state=1)
random_forest.fit(X_train, y_train)

/var/folders/zj/cjnnn8695qx8mqcc36hww0y80000gp/T/ipykernel_7857/3395526202.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest.fit(X_train, y_train)


In [ ]:
y_pred_train = random_forest.predict(X_train)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = random_forest.predict(X_val)
y_pred_val[y_pred_val < 0] = 0
 
print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))

## Experiment IV -- XGBoost

In [ ]:
xgb = xgboost.XGBRegressor(n_estimators=100, early_stopping_rounds=50, learning_rate=0.3)
xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=True)

In [ ]:
y_pred_train = xgb.predict(X_train)
y_pred_train[y_pred_train < 0] = 0
y_pred_val = xgb.predict(X_val)
y_pred_val[y_pred_val < 0] = 0

print("RMS log-error train: ", np.sqrt(mean_squared_log_error(y_train, y_pred_train)))
print("RMS log-error val: ", np.sqrt(mean_squared_log_error(y_val, y_pred_val)))

## Test (Moment of Truth)

In [ ]:
def main_predict(model, X_test):
    X_test_mod = X_test.copy()
    output = np.array([])
    start_day, end_day = X_test['day_of_month'].min(), X_test['day_of_month'].max()
        # we lost the dates, but we still have day_of_month, which is good enough for our experiment
        
    for day in range(start_day, end_day + 1):
        pred = model.predict(X_test_mod[X_test_mod['day_of_month'] == day])
        pred[pred < 0] = 0
        print(pred)
        output = np.concatenate([output, pred], axis=0)
        for future in range(day + 1, end_day + 1):
            X_test_mod.loc[X_test_mod[X_test_mod['day_of_month'] == future].index,
                           f'sales_lag_{(future - day):02d}'] = pred
            # fill out future values now that this sales figure is available
            
    return output

In [ ]:
y_pred_test = main_predict(lgb, X_test)

In [ ]:
delta_index = 3008016 - 3000888  # we inserted 4 Christmas days, 4 x 54 x 33 = 7128, which is the difference
submission = pd.DataFrame({'id': X_test.index - delta_index, 'sales': np.expm1(y_pred_test)})
submission.to_csv('submission.csv', index=False)

In [44]:
from keras.models import Sequential
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tqdm.keras import TqdmCallback

In [38]:
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

In [39]:
generator = TimeseriesGenerator(X_train_np, y_train_np, length=25, batch_size = 1)


In [41]:
 
print('%s => %s' % (generator[0][0], generator[0][1]))

[[[0.         0.91629076 0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.6931472  0.6931472  ... 0.         0.         0.        ]
  ...
  [0.         1.7917595  1.0986123  ... 0.         0.         0.        ]
  [0.         1.0986123  0.6931472  ... 0.         0.         0.        ]
  [0.         5.379091   5.3112087  ... 0.         0.         0.        ]]] => [[5.6811314]]


In [42]:
 
model = Sequential()    
    
    
model.add(LSTM(units = 50, activation='relu', return_sequences = True, input_shape = (25, len(X_train.columns)))) 
model.add(Dropout(0.3))
        
     
model.add(LSTM(units = 50,  activation='relu', return_sequences = True))                                    
model.add(Dropout(0.2))
        
    
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation = "linear"))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 25, 50)            48200     
                                                                 
 dropout_9 (Dropout)         (None, 25, 50)            0         
                                                                 
 lstm_12 (LSTM)              (None, 25, 50)            20200     
                                                                 
 dropout_10 (Dropout)        (None, 25, 50)            0         
                                                                 
 lstm_13 (LSTM)              (None, 50)                20200     
                                                                 
 dropout_11 (Dropout)        (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                

In [55]:
model.compile(optimizer='adam', loss='mean_squared_error')
#model.fit(generator, steps_per_epoch=len(generator), epochs=10, verbose=2)
 


In [58]:
model.fit(generator, steps_per_epoch=len(generator), epochs=10, batch_size=1, verbose=1)


Epoch 1/10
  12078/2255987 [..............................] - ETA: 9:23:08 - loss: 6.0883

KeyboardInterrupt: 

In [70]:
model = Sequential()
model.add(LSTM(units = 4, activation='relu', return_sequences = True, input_shape = (25, len(X_train.columns)))) 
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 25, 4)             3120      
                                                                 
 dense_11 (Dense)            (None, 25, 1)             5         
                                                                 
Total params: 3,125
Trainable params: 3,125
Non-trainable params: 0
_________________________________________________________________


In [71]:
history = model.fit(generator, steps_per_epoch=len(generator), epochs=25, batch_size=1, verbose=1)

Epoch 1/25
  11571/2255987 [..............................] - ETA: 3:11:10 - loss: 1279.5452